#### paper review assignment
# Attention Is All You Need

**리뷰 중점!  
Masked attention, Self attention, Multi head attention, Positional encoding, 사용하는 이유**

# 0. Abstract

Transformer는 attention 메커니즘만을 사용하여 높은 수준의 병렬화와 적은 학습 시간을 이루어낸 시퀀스 변환 모델이다. 해당 논문을 리뷰해보도록 하자.

# 1. Introduction

RNN은 주로 언어 모델링과 번역 task에서 많이 사용되고 있다. 하지만 hidden state $h_t$가 이전 시점 $h_{t-1}$을 입력으로 받는 순차연산의 특성으로 인해 병렬화가 불가하고, 입력 시퀀스가 길어질수록 성능이 떨어지는 단점이 있다.   
Attention 메커니즘이 이러한 문제의 대안으로 나왔으나 여전히 RNN과 함께 사용되는 정도였는데,  
이 논문에선 RNN 없이 attention 메커니즘만 사용한 아키텍처 Transformer를 소개한다.

# 2. Background

- Transformer에서 연산은 Extended Neural GPU, ByeNet, ConvS2S에 기반을 두어 일정 수준으로 축소시킬 수 있었다.  
- **Self-attention**은 자기 자신에게 attention을 수행하는 메커니즘이다. (문장 안에서 `대명사`의 뜻을 찾아내는 task를 예로 들면 이해가 편하다.)  
이는 독해나 요약과 같은 task에서 성공을 거두었고,  
→ Transformer는 self-attention만을 이용한 최초의 변환 모델이다. 

---

※ Attention 기초 개념 참고 그림 ※
![텍스트](https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG)
Query : 특정 시점의 디코더 은닉층  
Keys, Values : 모든 시점의 인코더 은닉층 키와 값  
> Attention이란, 내적을 통해 현재 디코더 영역이 어느 시점과 연관이 높은지를 찾아내는 메커니즘이라 할 수 있다.  

![텍스트](https://wikidocs.net/images/page/31379/transformer10.png)  
>Self-attention은 입력과 출력이 모두 자기 자신으로 독해와 비슷하다. 위 그림에서 it이 가리키는 걸 찾아내는  task를 예로 들 수 있다.

# 3. Model Architecture

기본적인 시퀀스 변환 모델은 $X_n$ --`encoder`→ $Z_n$ --`decoder`→ $Y_m$ 구조이다.  
Transformer는 **self-attention**과 **point-wise**, **FC layer**를 사용한다.

- Encoder : 6개의 layers로 이루어져있다. 각 layer에는 2개의 sub-layers가 있다.  
1) multi-head self-attention  
2) position-wise FC feed-forward  
두 layer 모두 residual connection(Skip connection)+Normalize, 즉 $LayerNorm(x + Sublayer(x))$하여 output값을 낸다.  
output 차원은 512로 통일시켰다.   


- Decoder : 6개의 layers로 이루어져있다. 각 layer에는 encoder와 동일한 sub-layers 두 개와 그 사이에 encoder의 output을 받는 multi-head self-attention까지 총 3개의 sub-layers가 있다. 역시나 encoder와 동일하게 각 sub-layer마다 residual connection과 normalization이 붙는다.  

    Decoder의 self-attention은 뒤 포지션에 attending하지 않도록 **masking**을 추가하였다. (문장은 끝까지 들어야 뜻을 알 수 있는데 아직 등장하지도 않은 단어로 번역하지 않기 위함이라 해야할까..? 뒤에서 더 살펴보자.)  

각 layer에 대한 자세한 설명은 뒤에서 하고, 우선 architecture를 도식화하면 다음과 같다.  

왼쪽이 encoder, 오른쪽이 decoder이고 이렇게 encoder-decoder layer가 6개 쌓여있다고 보면 된다.  
![텍스트](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FoWOJp%2FbtqAU5STV8C%2FeiUBrgA796JJviS2vLMx5K%2Fimg.png)  

### Attention  
위에서 설명했지만, attention은 query와 key-value를 가지고 가중합과 softmax를 거쳐 output을 낸다. 이제 Transformer에서 사용한 Attention에 대해 알아보자.   


#### ● Scaled Dot-Product Attention  
input은 $d_k$차원의 query와 key 그리고 $d_v$차원의 value, output은 $query * key /  \sqrt(d_k)$ 에 softmax를 취한다.  
(※ query * key 연산은 내적으로 어느 시점과 연관이 높은지 찾아내는 개념!)     
모든 쿼리에 동시에, 빠르게 attention을 수행하기 위해 행렬 형태로 연산이 들어간다. 식은 아래와 같다.   
$Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt d_k})V$  

가장 많이 사용되는 attention 함수로 dot-product attention과 addictive attention이 있는데, 차원이 커질수록 속도에 차이를 보인다. gradient vanishing 문제를 막기 위해 scaling을 해주었다.  
아래는 벡터로 도식화한 scaled dot-product attention이다.  
![텍스트](https://wikidocs.net/images/page/31379/transformer14_final.PNG)


#### ● Multi-Head Attention  
전체 차원에 attention을 한 번 수행하는 것이 아닌 **쪼개서 $h$번 수행한 후 다시 concat**하는 것을 말한다. query,key,value를 쪼개면 attention을 **병렬 수행**할 수 있고, 이는 정보를 여러 시각에서 볼 수 있게 하기 때문에 Transformer의 핵심 개념이다.  
Transformer에서는 $h=8$로 병렬화하여 $d_k=d_v=d_{model}/h=64$로 적용하였다. 

도식화에는 Multi-head Attention으로만 써있지만 하나의 attention 계산 방법으로 scaled dot-product attention을, 이를 병렬화시켜 multi-head attention을 적용하였다고 이해하면 되겠다. 

--- 

모델에서 attention이 어떻게 유기화되는지 살펴보면 다음과 같다.  
- query는 이전 decoder층에서 받고 key와 value는 encoder output에서 받으므로 decoder가 모든 인풋 시퀀스 시점에 접근할 수 있게 만든다.
- encoder에 self-attention이 있어 query,key,value를 이전 층의 encoder output에서 받으므로 encoder 역시나 모든 시점에 접근할 수 있게 만든다.
- decoder층의 self-attention은 **masking으로 다음 단어는 보지 못하도록 막아준다.**

![텍스트](https://paul-hyun.github.io/assets/2019-12-19/decoder_mask.png)

#### Position-wise Feed-Forward Networks  
ReLU와 선형식으로 이루어져있는데, 식은 다음과 같다.  

$FFN(x) = max(0,xW_1+b_1)W_2+b_2$   

시점별로는 동일하게 적용되나 층별로는 다르게 적용된다.

### Positional Encoding  
RNN이나 CNN을 사용하지 않으므로 시퀀스의 순서를 사용하기 위해 위치 정보를 담아두는 것을 말한다. 각 단어 임베딩 벡터에 위치 정보 positional encoding을 더해 모델에 입력한다. 도식화해보면 다음과 같다.  
![텍스트](https://wikidocs.net/images/page/31379/transformer6_final.PNG)  
이렇게 되면 같은 단어여도 자리에 따라 다르게 모델에 입력된다.

차원은 $d_{model}$로 동일하고, 식은 다음과 같다.  

$PE_(pos,2i) = sin(pos/10000^{2i/d_{model}})$  
$PE_(pos,2i+1) = cos(pos/10000^{2i/d_{model}})$, 
$pos$ : position, $i$ : 차원  

>sin과 cos 주기함수가 값이 동일해지는걸 방지해주기 때문에 겹치지 않고 position이 모델에 잘 입력될 것이라 본 것이다. 

>※ 위 식을 자세히 살펴보면, sin,cos의 범위는 일정하여 시퀀스 길이가 달라도 값이 튀지 않고, 시점 간의 거리가 같다. (즉, 1st와 2nd의 거리는 2nd와 3rd의 거리와 같다.)  
또한 $d_{model}$이 크므로 주기가 길어 겹칠 일이 드물다.

# 4. Why Self-Attention

**1. 각 layer의 연산량 감소**  
**2. 연산의 병렬화**   
**3. long-range 학습의 경로 길이**  
→ long-range 학습의 성능이 쟁점인데, forward와 backward 경로의 길이가 짧을수록 long-range 학습이 쉬워진다.

또한 self-attention은 모든 시점에 일정한 연산이 가해지므로 RNN보다 연산이 빠르다.

# 5.Training

- Training Data and Batching : 450만 문장을 byte-pair encoding하였고, 각 batch는 25000개의 source와 target tokens를 가진다.  
- Hardware and Schedule : 8 NVIDIA P100 GPUs에서 학습하였고, 각 step은 0.4초 소요하여 총 100,000 step을 12시간동안 학습하였다. 
- Optimizer : Adam optimizer에 $\beta_1 = 0.9, \beta_2 = 0.98, \epsilon = 10^{-9}$ 로 사용하였고,   
learning rate는 아래와 같은 식을 거쳐 사용하였다.  
$lr - d_{model}^{-0.5} * min(step\_num^{-0.5}, step\_num*warmup\_steps^{-1.5})$
- Regularization : Residual Dropout과 Label Smoothing을 적용하였다.

# 6. Results

English-to-German에서는 BLEU score 28.4, English-to-French에서는 BELU 41.0을 기록하였다.  
Transformer의 구성을 평가하기 위해 attention head와 key, value의 차원을 변경해본 결과, single attention 혹은 head가 과하게 많아지면 역시 성능이 떨어졌다. 최종 Transformer 모델은 적절한 hyperparameters의 결과이다.

# Conclusion

지금까지 attention 기반의 첫번째 시퀀스 변환 모델을 소개하였다.  
RNN이나 CNN 기반의 모델보다 빠르게 학습할 수 있었고, text에서 확장하여 이미지나 음성, 영상에 적용하는 데에 이어나가고자 한다.

# 참고자료

1. https://wikidocs.net/31379
2. https://reniew.github.io/43/
3. https://ynebula.tistory.com/51
4. https://paul-hyun.github.io/transformer-02/
5. https://pongdangstory.tistory.com/482

# 과제1 결과창

![텍스트](https://postfiles.pstatic.net/MjAyMDA5MjJfMjAx/MDAxNjAwNzYxOTk5MTEz.9cuR2jSqdt8oOOSZt1ifq8pHfgR26Y1RjSDgjs7f1AUg.LjeqIqlP2C4cMf-VS81A2N9V3FkG85G9z8fiUX-XCJog.PNG.tpdud0616/image.png?type=w773)